In [1]:
%load_ext autoreload
%autoreload 2

#load dataset
with open('input.txt', 'r') as f:
    text = f.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [2]:
print(len(text)) #dataset length

1115394


In [3]:
#identify vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
#create mapping of characters to integers and vice versa
stoi = dict((c, i) for i, c in enumerate(chars)) #string to int
itos = dict((i, c) for i, c in enumerate(chars)) #int to string
encode = lambda s: [stoi[c] for c in s] #encode string as list of ints
decode = lambda e: ''.join([itos[i] for i in e]) #decode list of ints to string

In [5]:
#torch initialization
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:
train_data, test_data = data[:int(0.9*len(data))], data[int(0.9*len(data)):] #90% train, 10% test
print(len(train_data), len(test_data))

1003854 111540


In [7]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1] #shifted by 1
for t in range(block_size):
    context = x[:t+1] #models goal is to predict next character given previous characters
    target = y[t] #target is next character
    print(f'for input {context} target is {target}')

for input tensor([18]) target is 47
for input tensor([18, 47]) target is 56
for input tensor([18, 47, 56]) target is 57
for input tensor([18, 47, 56, 57]) target is 58
for input tensor([18, 47, 56, 57, 58]) target is 1
for input tensor([18, 47, 56, 57, 58,  1]) target is 15
for input tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
for input tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [9]:
torch.manual_seed(1)
batch_size = 4 #number of sequences in a mini-batch
block_size = 8 #numbers of characters in a sequence

def get_batch(split):
    '''create a small batch of data from the text corpus'''
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #get random starting indices for sequences
    x = torch.stack([data[i:i+block_size] for i in ix]) #convert to 2D tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) #shifted by 1
    return x,y

x,y = get_batch('train')
print(x)
print(y)

tensor([[ 1, 51, 63,  1, 51, 39, 57, 58],
        [42,  1, 58, 59, 56, 52,  1, 58],
        [60, 47, 52, 43, 63, 39, 56, 42],
        [ 0,  0, 25, 13, 30, 21, 13, 26]])
tensor([[51, 63,  1, 51, 39, 57, 58, 43],
        [ 1, 58, 59, 56, 52,  1, 58, 46],
        [47, 52, 43, 63, 39, 56, 42,  1],
        [ 0, 25, 13, 30, 21, 13, 26, 13]])


In [10]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1)

class BigramLanguageModel(nn.Module):
    

    def __init__(self, vocab_size):
        super().__init__()
        #each token reads off logits for next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)


    def forward(self, idx, targets=None):
        '''
        idx: (B,T) tensor of ints
        targets: (B,T) tensor of ints
        '''
        logits = self.token_embedding_table(idx) #(B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets) #classification loss
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        '''
        idx: (B,T) array of indices in the current context
        '''
        for _ in range(max_new_tokens):
            logits, loss = self(idx) #get predictions
            logits = logits[:, -1, :] #focus on last timestep only. result is (B,C) tensor.
            probs = F.softmax(logits, dim=-1) #get probabilities for each class in vocabulary
            idx_next = torch.multinomial(probs, num_samples=1) #sample from distribution. result (B,1) tensor.
            idx = torch.cat([idx, idx_next], dim=-1) #append to context
        return idx

In [11]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(x, y)
print(logits.shape, loss)
generated_text = m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()
print(decode(generated_text))


torch.Size([32, 65]) tensor(4.7781, grad_fn=<NllLossBackward>)

-e$PyhE&YW$Hd.Z;mA:
3ANF!DG:'gIX,bHZ'yadMLRffLKKDQry3MvpSEcrHEkHapAD&nHNXlNYNcZNJVQ:AdIjZ;jJFz$hSY&C


In [12]:
#create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
batch_size=32
for step in range(5000):
    optimizer.zero_grad(set_to_none=True)
    x,y = get_batch('train')
    logits, loss = m(x,y)
    loss.backward()
    optimizer.step()
print(loss.item())

2.6214325428009033


In [14]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Til tXZ? r st s thiky,

Hean w:
FYor he bd P, sDour w'HERue?
DHaf YORorgritrepyowandu bquranglerexco t llDUColi3waswhenordk jor, bj! Sisht.
G me:
YXll;-Vaselp Lapo shvelorXbusengaryK:
Kil c,
Heyoutrdoure dxtstxVithuWAn rud:
ThBur-
rousoouerd S: 'dren be. '
Ourin s m,d tant.T?
TUzitinger'Whal:
Me havUMThe patRO:z, heardpudUD$XLAn herotWhe oo; Uve indXbey jutheayroremyopfouimaipis s tyeIAnth?
STURULowe. juCHUKA be we fStle mousods IFritnor hithomer dPSYont Par beste sis:
TreBew--thaIwo f bucatheFo


In [15]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1)

class LSTMLanguageModel(nn.Module):
    '''simple LSTM language model that predicts the next character.'''

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.lstm = nn.LSTM(vocab_size, 128, batch_first=True, num_layers=2)
        self.linear = nn.Linear(128, vocab_size)
    
    def forward(self, idx, targets=None):
        '''
        idx: (B,T) tensor of ints
        targets: (B,T) tensor of ints
        '''
        x = self.token_embedding_table(idx)
        x, _ = self.lstm(x)
        logits = self.linear(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        '''
        idx: (B,T) array of indices in the current context
        '''
        for _ in range(max_new_tokens):
            logits, _ = self(idx) #get predictions
            logits = logits[:, -1, :] #focus on last timestep only. result is (B,C) tensor.
            probs = F.softmax(logits, dim=-1) #get probabilities for each class in vocabulary
            idx_next = torch.multinomial(probs, num_samples=1) #sample from distribution. result (B,1) tensor.
            idx = torch.cat([idx, idx_next], dim=-1) #append to context
        return idx
    

In [16]:
lstm = LSTMLanguageModel(vocab_size)

#create optimizer
lstm_optimizer = torch.optim.AdamW(lstm.parameters(), lr=1e-3)
batch_size=32

In [17]:
#train
for step in range(5000):
    lstm_optimizer.zero_grad(set_to_none=True)
    x,y = get_batch('train')
    logits, loss = lstm(x,y)
    loss.backward()
    lstm_optimizer.step()
print(loss.item())

1.7586300373077393


In [18]:
print(decode(lstm.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



LUCIO:
I fay Honse: your may, prowsly.

MENENIUS:
In than yours deputing this not me; Tork.
Vorselder years, and gley,
Cricks, for your only: hear
Marrined, the did, are the kisnates and friends.
Doir fames. The grove a lie on your delisnemand, your chird
By othere, though art prespermaties in my more no friervatience, for scratter'd not trumble of him! well art trunked.

LUCIO:
Did I senful there chast an Like.

MENENIUS:
Whithie Those croves, you knot for flem trutchumently Trusiname?

QUEEN'
